In [1]:
%pip install -q cassio datasets langchain openai tiktoken

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import HuggingFaceEmbeddings


In [4]:
from datasets import load_dataset
import cassio

d:\langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
%pip install pyPDF2

Note: you may need to restart the kernel to use updated packages.


In [6]:
from PyPDF2 import PdfReader

In [7]:
ASTRA_DB_APPLICATION_TOKEN = "AstraCS:qOEBGMCvKPpkGBugwimsUilZ:fbd0420ce1581f2a7286dcd8b386097fa3edf0bf54d070ba6812546f568650a3"
ASTRA_DB_ID = "80419df6-d0c2-4f89-99b5-d7123ad7c8bd"

In [8]:
pdfreader = PdfReader("thecomedybible.pdf")

In [9]:
from typing_extensions import Concatenate
raw_text = ''
for i, page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text += content

In [10]:
raw_text

'Are you funny? Want to have a career in comedy? \nThis book can show you how to turn your sense of \nhumour into a money-making career—and \nthat\'s no joke! \nWhether you yearn to create a killer stand-up act, write \na sitcom, or be the star of your own one-person show, \nJudy Carter will help you develop your comedy skills and \nshow you how to make money from being funny. \nWritten in Carter\'s unique, take-no-prisoners voice, The \nComedy Bible is practical, inspirational and funny. Using \na hands-on workbook format, Judy Carter offers a series \nof day-by-day exercises drawn from her wide \nexperience as both a comic and comedy writer. Learn \nnot only how to write jokes, speeches and scripts, but \nalso where to sell them, how to pitch them, and even \nhow to negotiate a contract. Along with providing \nadditional \'insider\' tips from her celebrity friends, \nCarter shows you ways you can turn comedy into \ncash that you have never thought of before. \'Until comedians \ncan e

In [11]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [12]:
llm = Ollama(model="deepseek-r1:1.5b", temperature=0.1)
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


C:\Users\mohit\AppData\Local\Temp\ipykernel_13496\2256349389.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="deepseek-r1:1.5b", temperature=0.1)
C:\Users\mohit\AppData\Local\Temp\ipykernel_13496\2256349389.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L

In [13]:
astra_vector_store = Cassandra(
    embedding=embedding,
    session=None,
    keyspace=None,
    table_name="qa_mini_demo",
)


In [14]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)

texts = text_splitter.split_text(raw_text)

In [15]:
texts[:50]

["Are you funny? Want to have a career in comedy? \nThis book can show you how to turn your sense of \nhumour into a money-making career—and \nthat's no joke! \nWhether you yearn to create a killer stand-up act, write \na sitcom, or be the star of your own one-person show, \nJudy Carter will help you develop your comedy skills and \nshow you how to make money from being funny. \nWritten in Carter's unique, take-no-prisoners voice, The \nComedy Bible is practical, inspirational and funny. Using \na hands-on workbook format, Judy Carter offers a series \nof day-by-day exercises drawn from her wide \nexperience as both a comic and comedy writer. Learn \nnot only how to write jokes, speeches and scripts, but \nalso where to sell them, how to pitch them, and even",
 "experience as both a comic and comedy writer. Learn \nnot only how to write jokes, speeches and scripts, but \nalso where to sell them, how to pitch them, and even \nhow to negotiate a contract. Along with providing \nadditiona

In [16]:
astra_vector_store.add_texts(texts)
print("Inserted %i headlines." % len(texts))
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 882 headlines.


In [17]:
first_question = True
while True:
  if first_question:
      query_text = input("\nEnter your question(or type'quit' to exit):").strip()
  else:
    query_text = input("\nWhat's your next question(or type'quit' to exit):").strip()
  if query_text.lower() == "quit":
    break
  if query_text == "quit":
    continue
  first_question = False
  
  print("\nQUESTION:\"%s\"" % query_text)
  answer = astra_vector_index.query(query_text, llm=llm).strip()
  print("\nANSWER:\"%s\"\n" %answer)

  print("FIRST DOCUMENTS BY RELEVANCE:")
  for doc, score in astra_vector_store.similarity_search_with_score(query_text,k=4):
    print("[%0.4f] \"%s...\""%(score,doc.page_content))




QUESTION:""

ANSWER:"<think>
Okay, so I need to figure out the answer to this question based on the context provided. Let me read through the text carefully.

The text talks about DJs doing something and how listeners respond on morning shows. The target audience is adults 25-49. It mentions that if a DJ finds something funny, other people are likely to find it funny too. Then there's a quote from Sue Kolinsky, a comic and radio host, talking about her job as a DJ for Mason and Kolinsky Anti Show on WNEW FM 102.7.

Looking at the structure of the text, each paragraph seems to be a separate piece of information. The first part describes the target audience and how people respond when something is funny. Then there's a quote from Sue, who talks about her job and the show she worked on.

The question is asking for a helpful answer based on this context. Since the text provides information about the target audience and the response to funniness, I think the answer should reflect that. It 